In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time as time 
import pandas as pd 
import os 
import re 

In [3]:

# List of states & fct 
nigeria_states = ['ABIA','ADAMAWA','AKWA IBOM','ANAMBRA','BAUCHI','BAYELSA',
'BENUE','BORNO','CROSS RIVER','DELTA','EBONYI','EDO','EKITI','ENUGU','FCT',
'GOMBE','IMO','JIGAWA','KADUNA','KANO','KATSINA','KEBBI','KOGI','KWARA',
'LAGOS','NASARAWA','NIGER','OGUN','ONDO','OSUN','OYO','PLATEAU','RIVERS',
'SOKOTO','TARABA','YOBE','ZAMFARA']

In [2]:
def text_fider(website,text, exact=True):
    driver = webdriver.Chrome()

    # Navigate to the website
    driver.get(website)
    # Wait for the page to load
    time.sleep(3)

    # perform partial match
    if exact!=True:
        try:
            result = driver.find_element(By.PARTIAL_LINK_TEXT, text)
            # Extracting the link
            link = (result.get_attribute('href'))
        except:
            print('Unable to find :',text)
    # perform exact match
    else:
        try:
            result = driver.find_element(By.LINK_TEXT, text)
            # Extracting the link
            link = (result.get_attribute('href'))
        except:
            print('Unable to find :',text)
    return result.text,link

In [8]:

def result_clicker(website,polls,root_folder):
    for i in range(1,polls+1):
        driver = webdriver.Chrome()

        # Navigate to the website
        driver.get(website)
        # Wait for the page to load
        time.sleep(3)
        try:
            result = driver.find_element(By.XPATH, f'/html/body/app-root/div/app-activated/div/div/div/div/app-election-lga/div/div/div/div[2]/div[2]/div/div[2]/div[3]/div/div[{i}]/div/div[2]/button')
            # Extracting the link
             # Scroll down to bring the element into view
            driver.execute_script("arguments[0].scrollIntoView();", result)
            print(i,result)
            time.sleep(8)
            
            action = result.click()

            url = driver.current_url

            driver.get(url)
            time.sleep(3)
            iframe = driver.find_element(By.XPATH, '//iframe[contains(@src, "https://")]')
            driver.execute_script("arguments[0].scrollIntoView();", iframe)

            time.sleep(3)

            # Get the source URL of the embedded content
            src = iframe.get_attribute('src')
            #print(i,src)

            # Get the file type from the source URL
            file_type = src.split('.')[-1]

            # Download the content
            #location = f'D:/OneDrive/2023 Election Results/{root_folder}'
            # Create the root folder if it doesn't exist
            root_folder = re.sub(r'[^\w\s]', '', root_folder)
            root_folder = root_folder.replace(" ", "_")
            root_folder = root_folder.replace("/", "_")
            root_folder = root_folder.replace(":", "_")
            
            if not os.path.exists(root_folder):
                os.makedirs(root_folder)
            else:
                pass

            with open(f'D:/OneDrive/2023 Election Results/{root_folder}/{i}.{file_type}', "wb") as f:
                f.write(requests.get(src).content)

        except:
            print('No download at ',src)
            pass

In [9]:
def state_picker(state,loc_gov,ward,poll_unit = 60):
    driver = webdriver.Chrome()
    # Navigate to the website
    driver.get("https://www.inecelectionresults.ng/pres/elections/63f8f25b594e164f8146a213?type=pres")
    # Wait for the page to load
    time.sleep(3)
    result = driver.find_element(By.LINK_TEXT, state)
    link = (result.get_attribute('href'))

    local_governments = {}
    wards = {}
    polling = {}
    #create a dictionary of tuples (local govt, website)
    for i in range(1,loc_gov+1):
        try:
            #local government level
            output = text_fider(website=link,text=f'{i})', exact=False)
            local_governments[output] = None

            #ward level. search the local govt link for a ward
            for i in range(1,ward+1):
                try:
                    output_2 = text_fider(website=output[1],text=f'{i})', exact=False)
                    wards[output_2] = None
                    local_governments[output].append(output_2)
                except:
                    pass 
                
                #polling unit level. search the ward link for a polling unit
                try:
                    print(output_2[0])
                    result_clicker(website=output_2[1],polls=poll_unit, root_folder=output_2[0])
                except: pass 
        except:
            pass 
    return local_governments,wards, polling

In [11]:
Lagos = state_picker('LAGOS',loc_gov=1,ward=1,poll_unit=1)

ISALE/IDIMANGORO (Ward: 01) - LAGOS
1 <selenium.webdriver.remote.webelement.WebElement (session="a35f3027a83a0b1a813991fb04416b27", element="86ef15cc-7ada-4830-ae36-4cfd260425cf")>


In [40]:
Lagos[2]

{}